![](./figures/Logo.PNG)

## In this part of the tutorial, you will
* use a genetic algorithm to optimize model outputs globally
* critically assess the optimized parameters and objective function results
* implement and use your own objective function(s) 

- - -

# 2 d - Optimization

- - -

## 1 About optimization

In the previous tutorials, we have relied on our own capacity to tune the input parameters of our model. In this tutorial, we will use an algorithm to do this job for us by minimizing an objective function we define. Such algorithms run a model multiple times and determine the optimal combination of input parameters by computing the objective function for each model output. 

<div style="background:#e0f2fe; padding: 1%; border: 1mm solid SkyBlue">
    <h4><span>&#129300 </span>Your Turn I: Optimization Problems</h4>
    <p>Discuss with your neighbour what problems may occur when model optimization purely relies on an algorithm?</p>
</div>

## 2 Using global optimization

**Import packages**

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import random
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
import sys
sys.path.append('src/')
import HyMod
from ipywidgets import interact, Dropdown, IntText, GridBox, FloatRangeSlider

**Defining functions**

In [2]:
def bias(obs, sim):
    """
    Calculates bias
    obs :  The observed time series
    sim :  The simulated time series
    @returns : The bias
    """
    return np.mean(obs - sim)


def rmse(obs, sim):
    """
    Calculates root mean squared error
    obs :  The observed time series
    sim :  The simulated time series
    @returns : The RMSE
    """
    return np.sqrt(np.mean((obs - sim)**2))

def one_minus_nse(obs, sim):
    """
    Calculates Nash Sutcliffe efficiency
    obs :  The observed time series
    sim :  The simulated time series
    @returns : 1-NSE(obs, sim)
    """
    return 1-(1 - (np.sum((obs - sim)**2)) / (np.sum((obs - np.mean(obs))**2)))

def hymod_func(param, precip, pet, runoff_obs, objective_fun, n_days, info=False):
    param = np.array([param[0], param[1], param[2], 1/param[3], 1/param[4]])  # divide 1 by Rs and Rf for this HyMod implementation
    runoff_sim, states, fluxes = HyMod.hymod_sim(param, precip, pet)
    res = objective_fun(runoff_obs[n_days:], runoff_sim[n_days:])
    if info:
        print(f"Parameters: Sm = {param[0]}, beta = {param[1]}, alfa = {param[2]}, Rs = {param[3]}, Rf = {param[4]}")
        print(f"Result of objective function: {res}")
    return res

**Create and display interactive menus for selecting catchment**

In [3]:
# DO NOT ALTER! code to select the catchment

catchment_names = ["Siletz River, OR, USA", "Medina River, TX, USA", "Trout River, BC, Canada"]
dropdown = Dropdown(
    options=catchment_names,
    value=catchment_names[0],
    description='Catchment:',
    disabled=False
)

display(dropdown)

Dropdown(description='Catchment:', options=('Siletz River, OR, USA', 'Medina River, TX, USA', 'Trout River, BC…

**Read catchment data and prepare model input**

In [4]:
# Read catchment data
catchment_name = dropdown.value
# Read catchment data
file_dic = {catchment_names[0]: "camels_14305500", catchment_names[1]: "camels_08178880", catchment_names[2]: "hysets_10BE007"}
df_obs = pd.read_csv(f"data/{file_dic[catchment_name]}.csv")
# Make sure the date is interpreted as a datetime object -> makes temporal operations easier
df_obs.date = pd.to_datetime(df_obs['date'], format='%Y-%m-%d')
# Select time frame
start_date = '2002-10-01'
end_date = '2004-09-30'
spinup_days = 365

# Index frame by date
df_obs.set_index('date', inplace=True)
# Select time frame
df_obs = df_obs[start_date:end_date]
# Reformat the date for plotting
df_obs["date"] = df_obs.index.map(lambda s: s.strftime('%b-%d-%y'))
# Reindex
df_obs = df_obs.reset_index(drop=True)
# Select snow, precip, PET, streamflow and T
df_obs = df_obs[["snow_depth_water_equivalent_mean", "total_precipitation_sum","potential_evaporation_sum","streamflow", "temperature_2m_mean", "date"]]
# Rename variables
df_obs.columns = ["Snow [mm/day]", "P [mm/day]", "PET [mm/day]", "Q [mm/day]", "T [C]", "Date"]

# Prepare the data intput for both models
P = df_obs["P [mm/day]"].to_numpy()
evap = df_obs["PET [mm/day]"].to_numpy()
temp = df_obs["T [C]"].to_numpy()
q_obs = df_obs["Q [mm/day]"].to_numpy()

**Using global optmization**

The code below runs as sequence of three optimization trials with a low iteration count to show potential differences between the trials.

We use the **Differential Evolution** algorithm from `scipy` for optimization tasks.

Differential Evolution is a stochastic, population-based optimization algorithm. It works by evolving a population of candidate solutions over several iterations. The key steps are:

1. **Initialization**: Start with a randomly generated population of potential solutions.
2. **Mutation**: For each candidate, generate new candidate solutions by combining existing solutions using a mutation strategy.
3. **Crossover**: Create trial solutions by mixing mutated candidates with the current candidate.
4. **Selection**: Evaluate the fitness of trial solutions and select the best solutions to form the next generation.

The algorithm continues evolving the population until a stopping criterion is met, such as a maximum number of iterations or convergence to a solution.

In [11]:
# DO NOT ALTER! code to select the number of iterations, objective function and parameters

objective_functions = {"Bias":bias, "RMSE": rmse, "1-NSE": one_minus_nse, "Your OF 1": None, "Your OF 2": None}

def check_of_implemented():
    try:
        if dropdown_of.value in ["Your OF 1", "Your OF 2"]: 
            objective_functions[dropdown_of.value](np.array([]), np.array([]))
    except:
        raise NotImplementedError("NO WORRIES! Your objective functions seem to be not implemented yet or throws an error. Read the blue box below...") from None

max_iter_spn = IntText(
    value=5,
    description="Iterations:"
)

dropdown_of = Dropdown(
    options=objective_functions.keys(),
    value=list(objective_functions.keys())[0],
    description='Objective:',
    disabled=False
)

parameters = ["Sm", "beta", "alpha", "Rs", "Rf"]
bounds_rng = [(0,400), (0, 2), (0, 1), (2, 200), (1, 7)] # change max bounds range
bounds_rng = [FloatRangeSlider(value=bound, min=bound[0], max=bound[1], step=0.1, continous_update=False, readout_format=".1f", description=f"{param}:") for bound, param in zip(bounds_rng, parameters)]

GridBox([max_iter_spn, dropdown_of, *bounds_rng])

GridBox(children=(IntText(value=5, description='Iterations:'), Dropdown(description='Objective:', options=('Bi…

In [17]:
max_iter = max_iter_spn.value
bounds   = [bound_rng.value for bound_rng in bounds_rng]
OF       = objective_functions[dropdown_of.value]
check_of_implemented()    

print(f"Optimizing HyMOD for {catchment_name} with {dropdown_of.value} (iterating up to {max_iter} times)\n")
for cal_round in range(1, 4):
    # generate inital parameters randomly
    x0 = []
    for i in bounds:
        low, high = i
        x0.append(round(random.uniform(low,high), 2))
    print(f"{cal_round}. Trial")
    print(f"   Random intial parameters: Sm = {x0[0]}, beta = {x0[1]}, alfa = {x0[2]}, Rs = {x0[3]}, Rf = {x0[4]}")

    # run the genetic algorithm to optimize the parameters
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html
    # "bounds" are the parameter ranges
    # "args" are the arguements for the evaluated function (hymod_func)
    # "x0" is the initial guess or starting point for the optimization (the random parameters generated above)
    # "max_iter" is the maximum number of iterations we allow the opitmization algorithm to do
    hymod_calib = differential_evolution(hymod_func, bounds=bounds, args=(P, evap, q_obs, OF, spinup_days), x0=x0, maxiter=max_iter)
    parameters = hymod_calib.x  # extract parameters resulting from calibration
    
    # divide 1 by Rs and Rf for this HyMod implementation
    parameters = np.array([parameters[0], parameters[1], parameters[2], 1/parameters[3], 1/parameters[4]])
    parameters = np.round(parameters, decimals=3)  # round parameters to 3 decimals
    q_sim, states, fluxes = HyMod.hymod_sim(parameters, P, evap)  # run HyMod
    # calculate the result of the objective function for the optimized parameters
    obj_fun_result = OF(q_obs[spinup_days:], q_sim[spinup_days:])  
    
    # print calibration results
    parameters = np.array([parameters[0], parameters[1], parameters[2], 1/parameters[3], 1/parameters[4]])
    parameters = np.round(parameters, decimals=3)  # round parameters to 3 decimals
    print(f"   Optimization results:")
    print(f"     Best found parameter set: Sm = {parameters[0]}, beta = {parameters[1]}, alfa = {parameters[2]}, Rs = {parameters[3]}, Rf = {parameters[4]}")
    print(f"     (this is based on the best found result for {dropdown_of.value}: {round(obj_fun_result, 3)})\n")

Optimizing HyMOD for Siletz River, OR, USA with Your OF 1 (iterating up to 5 times)

1. Trial
   Random intial parameters: Sm = 245.25, beta = 0.54, alfa = 0.05, Rs = 156.33, Rf = 1.8
   Optimization results:
     Best found parameter set: Sm = 0.0, beta = 1.99, alfa = 1.0, Rs = 200.0, Rf = 1.152
     (this is based on the best found result for Your OF 1: 1.748)

2. Trial
   Random intial parameters: Sm = 394.2, beta = 1.49, alfa = 0.82, Rs = 86.03, Rf = 7.0
   Optimization results:
     Best found parameter set: Sm = 0.0, beta = 1.345, alfa = 1.0, Rs = 111.111, Rf = 1.152
     (this is based on the best found result for Your OF 1: 1.748)

3. Trial
   Random intial parameters: Sm = 264.61, beta = 1.92, alfa = 0.41, Rs = 156.96, Rf = 6.95
   Optimization results:
     Best found parameter set: Sm = 0.0, beta = 0.937, alfa = 1.0, Rs = 166.667, Rf = 1.153
     (this is based on the best found result for Your OF 1: 1.748)



<div style="background:#e0f2fe; padding: 1%; border: 1mm solid SkyBlue">
    <h4><span>&#129300 </span>Your Turn II: Using Global Optimization</h4>
    <ol>
        <li> Run the optimization for different catchments and different objective functions (change the variable "OF"). Compare the result of the different catchments! Where does the model perform better or worse (remember, what climatic regions the individual catchments represent)?</li>
        <li>Why do results of the optimized objective functions change between the trials?</li>
        <li>Why do, for the same results of the optimized objective function, some input parameters vary significantly?</li>
        <li>Write your own implementation of an objective function of your choice</li>
        <br>
        <p><i>Below, we have prepared two empty functions that are not yet implemented. You may choose any of the ones we used in previous tutorials or implement completely new ones. You can then use your new function with the optimization approach.</i></p>
    </ol>
</div>

In [16]:
def yourOF1(obs, sim):
    return bias(obs, sim) # TODO: remove and implement your own OF

def yourOF2(obs, sim):
    raise NotImplementedError() # TODO: remove and implement your own OF

# please do not remove this
objective_functions = {"Bias":bias, "RMSE": rmse, "1-NSE": one_minus_nse, "Your OF 1": yourOF1, "Your OF 2": yourOF2}